In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/Shareddrives/OPML/20210807_Titanic/彥嘉

[Errno 2] No such file or directory: '/content/drive/Shareddrives/OPML/Seminar/20210730_Titanic_廖家緯/彥嘉'
/content


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

from keras import backend as K 

In [ ]:
data_dir = "/content/drive/Shareddrives/OPML/Seminar/20210814_Mango_identification/data/mango_1st_stage/"
data = pd.read_csv(data_dir+"train.csv")
data.head()

,image_id,label
0,00002.jpg,C
1,00003.jpg,C
2,00004.jpg,B
3,00005.jpg,A
4,00007.jpg,A


In [ ]:
labelencoder_X = LabelEncoder()

In [ ]:
data_list = data.image_id.tolist()
# data_list

In [ ]:
data_label = data.label.tolist()
data_label = labelencoder_X.fit_transform(data_label)
label = np.zeros((data_label.shape[0],3))
label[np.arange(data_label.size),data_label]=1
print(data_label[:5])
print(label[:5])


[2 2 1 0 0]
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [ ]:
data_list_rename = [data_dir+"train/"+i for i in data_list]

In [ ]:
train=[]
for i in data_list_rename:
    img = load_img(i, color_mode="grayscale", target_size=(128, 128, 1))
    img_array = img_to_array(img)
    train.append(img_array)

In [ ]:
train = np.array(train,dtype=float)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
(x_train, x_test, y_train, y_test) = train_test_split(train, label, test_size=0.2, shuffle=True)
(x_val, x_test, y_val, y_test) = train_test_split(x_test, y_test, test_size=0.5)

In [ ]:
num_classes = 3

model = Sequential([
  layers.Conv2D(4, 5, input_shape=(128,128,1), padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.5),
  layers.Conv2D(16, 5, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.5),
  layers.Conv2D(32, 5, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.5),
  layers.Flatten(),
  layers.Dense(64, activation='relu'),
  layers.Dense(num_classes, activation='softmax')
])

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
# opt = tf.keras.optimizers.SGD(learning_rate=1e-3)
model.compile(optimizer=opt,
              loss=tf.keras.losses.CategoricalCrossentropy,
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 128, 128, 4)       104       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 64, 64, 4)         0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 64, 64, 4)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 64, 64, 16)        1616      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 32, 32, 16)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 32)       

In [ ]:
epochs=100
history = model.fit(
    x_train,
    y = y_train,
    batch_size = 1,
    validation_data = (x_test, y_test),
    epochs=epochs
)

Epoch 1/100


TypeError: ignored

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()